In [0]:
dbutils.secrets.listScopes()

In [0]:
df=dbutils.secrets.get(scope="youtube-db-scope",key="youtube-key-vault")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "d1df4878-b934-4df1-8357-06dd1a8ee374",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="youtube-db-scope",key="youtube-key-vault"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/fdc798aa-6f15-4e72-bf4a-6c813690fc75/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://youtube-data-analysis@adlsgen2youtube.dfs.core.windows.net/",
  mount_point = "/mnt/youtube-analysis-data",
  extra_configs = configs)

In [0]:
%fs
ls /mnt/youtube-analysis-data

In [0]:
%fs
ls dbfs:/mnt/youtube-analysis-data/transformed-data/statistics-reference-data/

In [0]:
statisticsReferenceDF=spark.read.format('parquet').load('/mnt/youtube-analysis-data/transformed-data/statistics-reference-data/*')
display(statisticsReferenceDF)

In [0]:
from pyspark.sql.functions import input_file_name, regexp_extract


In [0]:
statisticsReferenceDF=statisticsReferenceDF.withColumn('FileName', input_file_name())
display(statisticsReferenceDF)

In [0]:
statisticsReferenceDF=statisticsReferenceDF.withColumn('FileName', regexp_extract("FileName", "([^/]+$)", 0))
display(statisticsReferenceDF)

In [0]:
from pyspark.sql.functions import substring
statisticsReferenceDF=statisticsReferenceDF.withColumn('CountryCode', substring('FileName',1,2))
display(statisticsReferenceDF)

In [0]:
from pyspark.sql.functions import split
statisticsReferenceDF=statisticsReferenceDF.withColumn('Platform', split('kind','#')[0]).withColumn('Category',split('kind','#')[1])
display(statisticsReferenceDF)

In [0]:
statisticsReferenceDF=statisticsReferenceDF.drop('kind')
display(statisticsReferenceDF)

In [0]:
from pyspark.sql.functions import col
statisticsReferenceDF=statisticsReferenceDF.withColumn('id',col('id').cast('int'))
display(statisticsReferenceDF)

In [0]:
from pyspark.sql.functions import regexp_replace
statisticsReferenceDF=statisticsReferenceDF.withColumn('etag', regexp_replace('etag', '"', '')).withColumn('category', regexp_replace('category', 'Category', ''))
display(statisticsReferenceDF)

In [0]:
statisticsDF=spark.read.format('parquet').load('/mnt/youtube-analysis-data/transformed-data/raw-statistics/')
display(statisticsDF) 